In [1]:
import sys
import pandas as pd
import numpy as np
import requests
import string
import json
import datetime
import sqlite3
from pandas.core.ops import invalid_comparison

#web
import warnings
warnings.filterwarnings('ignore')

import ipywidgets as widgets

from IPython.display import display, clear_output

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, Layout, Button, HBox, VBox, Box

In [2]:
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
!jupyter serverextension enable voila --sys-prefix

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
Enabling: voila
- Writing config: /Users/yatingyang/opt/anaconda3/etc/jupyter
    - Validating...
Error loading server extension voila
      X is voila importable?


# EDFS

In [3]:
edfs = sqlite3.connect("edfs.db")
cur = edfs.cursor()

In [4]:
# SQLite set up
edfs = sqlite3.connect("edfs.db")
cur = edfs.cursor()

cur.execute("CREATE TABLE meta(id INTEGER primary key, name TEXT, type TEXT)") 
cur.execute("INSERT INTO meta VALUES (1,'/','d')") # root dir
# meta table stores the id for each file and dir
# Do we need to store permision, access and modification time?
# assign id in order, use max()+1 to assign the id for the next dir/file
cur.execute("CREATE TABLE dir(parent INTEGER, child INTEGER, primary key (parent, child))") 
# dir table stores the directory structure of the file system
cur.execute("CREATE TABLE par(id INTEGER, location)") 
# par table stores the partition infor about a file


In [5]:
def exist(path): # path: directory or file, eg. "/user/john" or "/user/hello.txt"
    path = path.strip() # remove white space before or after the path
    if path[0] == "/" and path[-1] != "/":# valid path starts with "/" - root, and does not end with "/"
    #id_parent = cur.execute("select id from meta where name = '/'").fetchall()  id of root
        id_parent = 1
        path = path[1:] # get rid of the root
        path = path.split("/") # list of the rest of the path - the name of each directory/file
        n = len(path)
        while n > 0: 
            position = len(path)-n
            id_child = cur.execute("select child from dir where parent = ? and child in (select id FROM meta where name = ?)", (id_parent, path[position])).fetchall()
          # check if the child exist and if the the parent-child relationship exist
            if id_child != []:# if there exists such child, it should be unique
                id_parent = id_child[0][0] # set the child as the new parent, and go back to the while loop to check for the next directory/file in path
                n = n - 1
            else:
                break
        if n > 0:
            return ""
        else: 
            return id_parent
    elif path == "/": # if the input is the root
        return 1
    else: 
        return ""

In [6]:
def mkdir(d):# d: directory
  # if the directory already exists, notify the user
  # check if the dir exist: get the id for the parent dir, search it in the dir table to get all the children
  # use a for loop to go through every child's name in meta table using the ids
    d = str(d)
    if d == "/":
        print("Directory already exist.")
    elif d[0] == "/" and d[-1] != "/":
        d = d.split('/')
        d_new = d # copy the list of directories into d_new
        d_new = d_new[:-1]
        if d_new == ['']: 
            d_new = "/"
        else: 
            d_new = "/".join(d_new)
        id_parent = exist(d_new)
        if id_parent != "": 
            type_parent = cur.execute("select type from meta where meta.id = ?", (id_parent,)).fetchone()[0]
            if type_parent == "d": 
                id_child = cur.execute("select child from dir where parent = ? and child in (select id FROM meta where name = ?)", (id_parent, d[-1])).fetchall()
                if id_child == []: # check if the directory already exist in the system
                    max_id = cur.execute("SELECT max(id) FROM meta").fetchone()
                    max_id = max_id[0]
                    id_child = max_id + 1 # the id assigned to the new directory
                    cur.execute("INSERT INTO meta VALUES (?,?,'d')",(id_child, d[-1])) ##id needs to be defined
                    cur.execute("INSERT INTO dir VALUES (?,?)",(id_parent,id_child))
                    print("The directory has been created successfully.")
                else: 
                    print("Directory already exist.")
            else: 
                print("Please enter a valid directory.")
        else: 
            print("Please enter a valid directory.")
    else:
        print("Please enter a valid directory.")

In [7]:
mkdir_text = widgets.Text(value = '',
                          placeholder ='type the directory here',
                          description = 'mkdir',
                          disabled = False)
mkdir_instruction = widgets.widgets.HTML('Create a directory in file system. Instruction: enter the directory in the box, e.g. mkdir /user, and click on submit.')

submit_button = widgets.Button(description='Submit')
def clicked(*b):
    output1.clear_output()
    with output1:
        mkdir(mkdir_text.value)
output1 = widgets.Output()

layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    width='70%')
mkdir_items = [mkdir_text, submit_button]
box = Box(children = mkdir_items, layout = layout)
display(mkdir_instruction, box, output1)
submit_button.on_click(clicked)

HTML(value='Create a directory in file system. Instruction: enter the directory in the box, e.g. mkdir /user, …

Box(children=(Text(value='', description='mkdir', placeholder='type the directory here'), Button(description='…

Output()

In [8]:
def ls(d):
    # d: directory
    d = str(d)
    if d == "/":
        l = cur.execute("select name from meta where id in (select child from dir where parent = 1)").fetchall()
        l_new = []
        for i in range(len(l)):
            l_new.append(l[i][0])
            #print(l[i][0])
        return l_new
    #return cur.execute("select name from meta where id in (select child from dir where parent = 1)").fetchall()
    else: 
        d_new = d # copy the list of directories into d_new
        d_new = d_new.split('/')
        id_parent = exist(d)
        if id_parent != "":
            l = cur.execute("select name from meta where meta.id in (select child from dir where parent = ?)",(id_parent,)).fetchall()
            l_new = []
            for i in range(len(l)):
                l_new.append(l[i][0])
                #print(l[i][0])
            return l_new
        #return cur.execute("select name from meta where meta.id in (select child from dir where parent = ?)",(id_parent,)).fetchall()
        else: 
            print("Please enter a valid directory.")

In [9]:
ls_text = widgets.Text(value = '',
                          placeholder ='type the directory here',
                          description = 'ls',
                          disabled = False)
ls_instruction = widgets.widgets.HTML('Listing content of a given directory. Instruction: enter the directory in the box, e.g. ls /user, and click on submit.)')

submit_button = widgets.Button(description='Submit')
def clicked(*b):
    output2.clear_output()
    with output2:
        list = ls(ls_text.value)
        try: 
            for i in list: 
                print(i)
        except: 
            pass
output2 = widgets.Output()

ls_items = [ls_text, submit_button]
box = Box(children = ls_items, layout = layout)
display(ls_instruction, box, output2)
submit_button.on_click(clicked)

HTML(value='Listing content of a given directory. Instruction: enter the directory in the box, e.g. ls /user, …

Box(children=(Text(value='', description='ls', placeholder='type the directory here'), Button(description='Sub…

Output()

In [10]:
def put(file, path, k): # file: file name, path: the directory storing the file, k: # of partitions
    file = str(file)
    path = str(path)
    k = int(k)
    if file[-4:] == ".csv": 
        check = cur.execute("select id FROM meta where name = ? and type = 'f'", (file,)).fetchall()
        if check == []: # if the file does not exist
            id_parent = exist(path)
            if id_parent != "":
                try: 
                    df = pd.read_csv(file)

                    max_id = cur.execute("SELECT max(id) FROM meta").fetchone()
                    max_id = max_id[0]
                    fid = max_id + 1 # the id assigned to the new directory
                    cur.execute("INSERT INTO meta VALUES (?,?,'f')",(fid, file))
                    cur.execute("INSERT INTO dir VALUES (?,?)",(id_parent, fid))

                    size = len(df.index)//k # partition size for first k-1 partitions
                    start = 0
                    fname = file.split(".")[0]
                    for i in range(1, k): # the first k-1 partitions
                        cur.execute("INSERT INTO par VALUES (?,?)",(fid, fname+str(i))) # information on partition
                        stop = start + size - 1
                        partition = df.loc[start:stop]
                        partition.to_sql(name = fname+str(i), con = edfs, index = False) ##### cannot retrieve the created table
                        start = stop + 1
                    cur.execute("INSERT INTO par VALUES (?,?)",(fid, fname+str(k)))
                    partition = df.loc[start:]
                    partition.to_sql(name = fname+str(k), con = edfs, index = False) ##### cannot retrieve the created table
                    print("The file has been uploaded successfully.")
                except: 
                    print("There is no such file.")
            else:
                print("Please enter a valid directory.")
        else:
            print("This file already exist in the system.")
    else: 
        print("Sorry, we only support uploading .csv files. ")

In [11]:
style = {'description_width': 'initial'}
put_file = widgets.Text(value = '',
                          placeholder ='type the file name here',
                          description = 'put',
                          disabled = False)
put_path = widgets.Text(value = '',
                          placeholder ='type the directory here',
                          description = '',
                          disabled = False)
put_k = widgets.IntText(description = '',
                        disabled = False,
                       style = style)
put_instruction = widgets.widgets.HTML('Uploading a file to file system. Instruction: enter the file name in the left box, enter the directory you want to put the file into in the second box, and enter number of partitions you want to store the file in, e.g., put(cars.csv, /user/john, 5), and click on submit.')

submit_button = widgets.Button(description='Submit')
def clicked(*b):
    output5.clear_output()
    with output5: 
        put(put_file.value, put_path.value, put_k.value)
        
output5 = widgets.Output()

put_items = [put_file, put_path, put_k, submit_button]
box = Box(children = put_items, layout = layout)
display(put_instruction, box, output5)
submit_button.on_click(clicked)


HTML(value='Uploading a file to file system. Instruction: enter the file name in the left box, enter the direc…

Box(children=(Text(value='', description='put', placeholder='type the file name here'), Text(value='', placeho…

Output()

In [12]:
def cat(file): # file: must include file path
    file = str(file)
    if exist(file) != "": 
        fid = exist(file)
        t = cur.execute("select type from meta where meta.id = ?", (fid,)).fetchone()
        t = t[0]
        if t == 'f': 
            loc = cur.execute("select location from par where par.id = ?", (fid,)).fetchall()
            df = pd.DataFrame()
            for l in loc:
                df_new = pd.read_sql('select * from ' + l[0], edfs)
                df = pd.concat([df, df_new], axis=0)
            print(df)
        else: 
            print("Please enter a valid file.")
    else:
        print("Please enter a valid file.")

In [13]:
cat_text = widgets.Text(value = '',
                          placeholder ='type the file path here',
                          description = 'cat',
                          disabled = False)
cat_instruction = widgets.widgets.HTML('Display content of a file. Instruction: enter the file path in the box, e.g. cat /user/john/hello.txt, and click on submit.')

submit_button = widgets.Button(description='Submit')
def clicked(*b):
    output3.clear_output()
    with output3: 
        cat(cat_text.value)
output3 = widgets.Output()

cat_items = [cat_text, submit_button]
box = Box(children = cat_items, layout = layout)
display(cat_instruction, box, output3)
submit_button.on_click(clicked)

HTML(value='Display content of a file. Instruction: enter the file path in the box, e.g. cat /user/john/hello.…

Box(children=(Text(value='', description='cat', placeholder='type the file path here'), Button(description='Su…

Output()

In [14]:
def getPartitionLocations(file): # file: including path
    file = str(file)
    if exist(file) != "": 
        fid = exist(file)
        t = cur.execute("select type from meta where meta.id = ?", (fid,)).fetchone()
        t = t[0]
        if t == 'f': 
            loc = cur.execute("select location from par where par.id = ?", (fid,)).fetchall()
            L = []
            for i in loc: 
                L.append(i[0])
            return L
        else: 
            print("Please enter a valid file.")
    else: 
        print("Please enter a valid file.")

In [15]:
gPL_text = widgets.Text(value = '',
                          placeholder ='type the file path here',
                          description = 'getPartitionLocations',
                          disabled = False, 
                       style = style)
gPL_instruction = widgets.widgets.HTML('Return the locations of partitions of the file, in this case, the locations are table names. Instruction: enter the file path in the box, e.g. getPartitionLocations /user/john/hello.txt, and click on submit.')

submit_button = widgets.Button(description='Submit')
def clicked(*b):
    output6.clear_output()
    with output6: 
        Loc = getPartitionLocations(gPL_text.value)
        if Loc != None: 
            print(Loc)
output6 = widgets.Output()

gPL_items = [gPL_text, submit_button]
box = Box(children = gPL_items, layout = layout)
display(gPL_instruction, box, output6)
submit_button.on_click(clicked)

HTML(value='Return the locations of partitions of the file, in this case, the locations are table names. Instr…

Box(children=(Text(value='', description='getPartitionLocations', placeholder='type the file path here', style…

Output()

In [16]:
def readPartition(file, num): # file: including path, num: partition #
    file = str(file)
    num = int(num)
    if exist(file) != "": 
        fid = exist(file)
        t = cur.execute("select type from meta where meta.id = ?", (fid,)).fetchone()
        t = t[0]
        if t == 'f': 
            if num > 0 and num <= len(getPartitionLocations(file)):
                loc = cur.execute("select location from par where par.id = ?", (fid,)).fetchall()
                #df = pd.DataFrame()
                df_new = pd.read_sql('select * from ' + loc[num-1][0], edfs)
                return df_new
            else: 
                print("Please enter a valid partition#.")
        else: 
            print("Please enter a valid file.")
    else:
        print("Please enter a valid file.")

In [17]:
rP_file = widgets.Text(value = '',
                          placeholder ='type the file path here',
                          description = 'readPartition',
                          disabled = False,
                      style=style)
rP_num = widgets.IntText(placeholder ='type partition# here',
                          description = '',
                          disabled = False)
rP_instruction = widgets.widgets.HTML('Show the content of a particular partition of the specified file. Instruction: enter the file path in the left box and specify the partition # you want to read in the right box, e.g. readPartition(/wildfire.csv, 3), and click on submit.')

submit_button = widgets.Button(description='Submit')
def clicked(*b):
    output7.clear_output()
    with output7: 
        Par = readPartition(rP_file.value, rP_num.value)
        print(Par)
output7 = widgets.Output()

rP_items = [rP_file,rP_num, submit_button]
box = Box(children = rP_items, layout = layout)
display(rP_instruction, box, output7)
submit_button.on_click(clicked)

HTML(value='Show the content of a particular partition of the specified file. Instruction: enter the file path…

Box(children=(Text(value='', description='readPartition', placeholder='type the file path here', style=Descrip…

Output()

In [18]:
def rm(file): 
    file = str(file)
    fid = exist(file)
    if fid != "":
        t = cur.execute("select type from meta where meta.id = ?", (fid,)).fetchone()
        t = t[0]
        if t == 'f':
            d = file
            d = d.split('/')
            d = d[:-1]
            if d == ['']: 
                d = "/"
            else:
                d = "/".join(d)
            id_parent = exist(d)
            # delete the file information in meta, parent-child relationship in dir, and the partition tables in par
            table = cur.execute("select location from par where id = ?", (fid,)).fetchall()
            for x in table:
                tname = x[0]
                cur.execute("drop table " + tname) # go thru each location in par table, and drop all the table
            cur.execute("delete from par where id = ?", (fid,))
            cur.execute("delete from meta where id = ?", (fid,))
            cur.execute("delete from dir where parent = ? and child = ?", (id_parent, fid))
            print("The file has been removed successfully.")
        else: #if it's a file
            print("Please enter a valid file.")
    else: 
        print("Please enter a valid file.")

In [19]:
rm_text = widgets.Text(value = '',
                          placeholder ='type the file path here',
                          description = 'rm',
                          disabled = False)
rm_instruction = widgets.widgets.HTML('Remove a file from the file system. Instruction: enter the file path in the box, e.g. rm /user/john/hello.txt, and click on submit.')

submit_button = widgets.Button(description='Submit')
def clicked(*b):
    output4.clear_output()
    with output4: 
        rm(rm_text.value)
output4 = widgets.Output()

rm_items = [rm_text, submit_button]
box = Box(children = rm_items, layout = layout)
display(rm_instruction, box, output4)
submit_button.on_click(clicked)

HTML(value='Remove a file from the file system. Instruction: enter the file path in the box, e.g. rm /user/joh…

Box(children=(Text(value='', description='rm', placeholder='type the file path here'), Button(description='Sub…

Output()

# Navigate through EDFS

In [20]:
def navigate(button): 
    clear_output()
    if button.description[0] == "/": 
        res = ls(button.description)
        back = Button(description = "go back to:"+button.description)
        for r in res: 
            if button.description == "/": 
                globals()["button"+r] = Button(description = button.description+str(r))
            else: 
                globals()["button"+r] = Button(description = button.description+"/"+str(r))
            globals()["button"+r].on_click(navigate)
            display(globals()["button"+r])
        back.on_click(navigate)
        display(back)
    else: 
        d = button.description
        d = d.split(':')
        d = d[1]
        if d == "/": 
            display(root)
        else: 
            d = d.split('/')
            d = d[:-1]
            if d == ['']: 
                d = "/"
            else:
                d = "/".join(d)
            res = ls(d)
            for r in res: 
                if d == "/": 
                    globals()["button"+r] = Button(description = d+str(r))
                else: 
                    globals()["button"+r] = Button(description = d+"/"+str(r))
                globals()["button"+r].on_click(navigate)
                display(globals()["button"+r])
            back = Button(description = "go back to:"+d)
            back.on_click(navigate)
            display(back)
root = Button(description = "/")
root.on_click(navigate)
display(root)

Button(description='/', style=ButtonStyle())

# Search and Analytics

In [21]:
def getpath(filename): 
    id_file = cur.execute("select id from meta where name = ?", (filename,)).fetchone()[0]
    def get(idn, path): # input is name(no path), but files are unique in the system
        id_parent = cur.execute("select parent from dir where child = ?",(idn,)).fetchone()[0]
        name_parent = cur.execute("select name from meta where meta.id = ?",(id_parent,)).fetchone()[0]

        if name_parent == "/":
            path = "/" + path
            return path

        path = name_parent + '/' + path
        path = get(id_parent, path)
        return path
    return get(id_file, filename)

In [22]:
#search by acres burned, output the wildfire name
def search_wildfire(l, h):
    l = int(l)
    h = int(h)
    L = []
    path = getpath("wildfire.csv")
    for i in range(len(getPartitionLocations(path))):
        df = readPartition(path, i+1)
        for index in df.index:
            if (df["AcresBurned"][index] >= min(l,h)) and (df["AcresBurned"][index] <= max(l,h)):
                L.append(df["Name"][index])
    L_new = []
    for i in range(len(L)):
        if L[i] not in L_new:
            L_new.append(L[i])
    print("The result is:", L_new)

In [23]:
def Mapper_search(l, h, df):
    l = int(l)
    h = int(h)
    L = []
    for index in df.index:
        if (df["AcresBurned"][index] >= min(l,h)) and (df["AcresBurned"][index] <= max(l,h)):
            L.append(df["Name"][index])
    return L
    
def Reducer_search(L):
    L_new = []
    for i in range(len(L)):
        if L[i] not in L_new:
            L_new.append(L[i])
    print(L_new)
    

In [24]:
search_lower = widgets.IntText(placeholder ='type here',
                          description = 'Range:',
                          disabled = False,
                      style=style)
search_higher = widgets.IntText(placeholder ='type here',
                          description = '',
                          disabled = False,
                        style=style)
search_instruction = widgets.widgets.HTML('Search names of California wildfires happened in 2018 by number of Acres burned:')
sql_ins=widgets.widgets.HTML('select unique Name, from wildfire, where AcresBurned > (lower) and AcresBurned < (upper)')
button_layout = widgets.Layout(width='auto', height='40px')
submit_button = widgets.Button(description='Submit')
def clicked(*b):
    output8.clear_output()
    with output8: 
        search_wildfire(search_lower.value, search_higher.value)
        print("This search function takes a range defined by lower and upper for acres burned as input. It first uses mapPartition function on each partition of the wildfire dataset to get all the wildfires names within the given range of acres burned. Then the reduce function takes the mapPartition result for all partitions as input to check whether there are repeated names. The reduce function returns a list of unique wildfires names within the given range of acres burned.")
        path = getpath("wildfire.csv")
        r = range(len(getPartitionLocations(path)))
        print("Input for mapPartition function:" )
        for i in r:
            print("Partiton #"+str(i+1)+":")
            print(readPartition(path, i+1))
        L = []
        print("Output for mapPartition function:" )
        for i in r:
            print("Partiton #"+str(i+1)+":")
            Par = readPartition(path, i+1)
            M = Mapper_search(search_lower.value, search_higher.value, Par)
            print(M)
            L = L + M
        print("Input for reduce function:\n", L)
        print("Output for reduce function:")
        Reducer_search(L)
            
output8 = widgets.Output()

search_items = [search_lower,search_higher, submit_button]
box = Box(children = search_items, layout = layout)
display(search_instruction,sql_ins, box, output8)
submit_button.on_click(clicked)

HTML(value='Search names of California wildfires happened in 2018 by number of Acres burned:')

HTML(value='select unique Name, from wildfire, where AcresBurned > (lower) and AcresBurned < (upper)')

Box(children=(IntText(value=0, description='Range:', style=DescriptionStyle(description_width='initial')), Int…

Output()

In [25]:
# select Region, (user_input) from temp, where month = (user_input)
def search_weather(choice, month): # input: weather index and region
    choice = str(choice)
    month = str(month)
    path = getpath("weather.csv")
    Mapper = {} # list for storing dictionaries (key, value) pairs
    for i in range(len(getPartitionLocations(path))): # i+1
        Par = readPartition(path, i+1)
        Reg = Par["Region"].to_numpy().tolist()
        Mon = Par["Months"].to_numpy().tolist()
        Cho = Par[choice].to_numpy().tolist()
        for i in range(len(Mon)): 
            if Mon[i] == month: 
                Mapper[Reg[i]] = Cho[i]
    print("The result is:", Mapper)

In [26]:
def Mapper_sw(choice, month, Par):
    Mapper = {}
    Reg = Par["Region"].to_numpy().tolist()
    Mon = Par["Months"].to_numpy().tolist()
    Cho = Par[choice].to_numpy().tolist()
    for i in range(len(Mon)): 
        if Mon[i] == month: 
            Mapper[Reg[i]] = Cho[i]
    print(Mapper)

In [27]:
sw_month = widgets.Dropdown(
    options=['Jan', 'Feb', 'Mar', 'Apr', 'May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'],
    description='Month:',
    disabled=False,
    style=style
)
sw_choice = widgets.Dropdown(
    options=['Precipitation(in)', 'AvgTemperature(F)', 'AvgHumidity(%)'],
    description='Factor:',
    disabled=False,
    style=style
)
sw_instruction = widgets.widgets.HTML('Show the data of a specified weather factor that relates to California wildfires in a specified month in 2018 in each regions:')
sw_sql = widgets.widgets.HTML('select counties, (choice) from weather where month = (month)')
submit_button = widgets.Button(description='Submit')
def clicked(*b):
    output9.clear_output()
    with output9: 
        search_weather(sw_choice.value, sw_month.value)
        print("This search function takes a user’s choice of weather data type and a month as input. It uses mapPartition function on each partition of the weather dataset to get all the weather data for each region in the given month, and outputs the full dictionary of regions corresponding to their weather data. ")
        path = getpath("weather.csv")
        r = range(len(getPartitionLocations(path)))
        print("Input for mapPartition function:" )
        for i in r:
            print("Partiton #"+str(i+1)+":")
            print(readPartition(path, i+1))
        print("Output for mapPartition function:" )
        for i in r:
            print("Partiton #"+str(i+1)+":")
            Par = readPartition(path, i+1)
            Mapper_sw(sw_choice.value, sw_month.value, Par)
output9 = widgets.Output()
display(sw_instruction,sw_sql, sw_month, sw_choice, submit_button, output9)
submit_button.on_click(clicked)

HTML(value='Show the data of a specified weather factor that relates to California wildfires in a specified mo…

HTML(value='select counties, (choice) from weather where month = (month)')

Dropdown(description='Month:', options=('Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', …

Dropdown(description='Factor:', options=('Precipitation(in)', 'AvgTemperature(F)', 'AvgHumidity(%)'), style=De…

Button(description='Submit', style=ButtonStyle())

Output()

In [28]:
# select Counties, count(Name) from wildfire, group by Counties, having count > ?
def analyze_wildfire(l, u, m): # output: counties with # of wildfires >= m
    l = int(l)
    u = int(u)
    m = int(m)
    path = getpath("wildfire.csv")
    Mapper = [] # list for storing dictionaries key, value pairs
    for i in range(len(getPartitionLocations(path))): # i+1
        Par = readPartition(path, i+1)
        Coun = Par['Counties'].to_numpy().tolist()
        Month = Par['Started'].to_numpy().tolist()
        for i in range(len(Coun)):
            if int(Month[i][5:7]) >= min(l,u) and int(Month[i][5:7]) <= max(l,u): 
                D = {} # dictionary for storing each (county,1) pair
                D[Coun[i]] = 1
                Mapper.append(D)
    #print(Mapper)
    Comb = {} # combiner 
    for d in Mapper:
        county = list(d.keys())[0]
        if county not in Comb:
            Comb[county] = [1]
        else:
            Comb[county].append(1)
    #print(Comb)
    Reducer = {}
    for key in Comb:
        if sum(Comb[key]) >= m:
            Reducer[key] = sum(Comb[key])
    Reducer = sorted(Reducer.items(), key=lambda x:x[1], reverse = True)
    Reducer = dict(Reducer)
    print("The result is:",Reducer)

In [29]:
def Mapper_aw(Par,l,u): 
    l = int(l)
    u = int(u)
    Mapper = []
    Coun = Par['Counties'].to_numpy().tolist()
    Month = Par['Started'].to_numpy().tolist()
    for i in range(len(Coun)):
        if int(Month[i][5:7]) >= min(l,u) and int(Month[i][5:7]) <= max(l,u): 
            D = {} # dictionary for storing each (county,1) pair
            D[Coun[i]] = 1
            Mapper.append(D)
    return Mapper
def Reducer_aw(Mapper, m):
    m = int(m)
    Comb = {} # combiner 
    for d in Mapper:
        county = list(d.keys())[0]
        if county not in Comb:
            Comb[county] = [1]
        else:
            Comb[county].append(1)
    Reducer = {}
    for key in Comb:
        if sum(Comb[key]) >= m:
            Reducer[key] = sum(Comb[key])
    Reducer = sorted(Reducer.items(), key=lambda x:x[1], reverse = True)
    Reducer = dict(Reducer)
    print(Reducer)

In [30]:
aw_lower = widgets.Dropdown(
    options=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    description='Month range:',
    disabled=False,
    style=style
)
aw_upper = widgets.Dropdown(
    options=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    description='',
    disabled=False,
    style=style
)
aw_num = widgets.IntText(description = 'number:',
                        disabled = False,
                       style = style)
aw_instruction = widgets.widgets.HTML('Show a list of counties in California that had more than the given number of wildfires during the given range of months in 2018:')
aw_sql = widgets.widgets.HTML('select Counties, count(Name) from wildfire where month >= (lower) and month <= (upper) group by Counties having count > (num) order by count(Name) desc')
submit_button = widgets.Button(description='Submit')
def clicked(*b):
    output10.clear_output()
    with output10: 
        analyze_wildfire(aw_lower.value, aw_upper.value, aw_num.value)
        print("This analytics function takes a range defined by lower and upper for months and a number as input. It first uses mapPartition function on each partition of the wildfire dataset to get all the counties in which the wildfires started within the given range of months. The mapPartition function returns county names with a corresponding count 1. Then the reduce function takes the mapPartition result for all partitions as input to group counties together and sum up the counts. If the count is greater than the num given, the reduce function will return the county name with the count. The output for the reduce function is a dictionary of county names with corresponding count of wildfires which happened in the given range of month and which is greater than the num given. ")
        path = getpath("wildfire.csv")
        r = range(len(getPartitionLocations(path)))
        print("Input for mapPartition function:" )
        for i in r:
            print("Partiton #"+str(i+1)+":")
            print(readPartition(path, i+1))
        L = []
        print("Output for mapPartition function:" )
        for i in r:
            print("Partiton #"+str(i+1)+":")
            Par = readPartition(path, i+1)
            M = Mapper_aw(Par, aw_lower.value, aw_upper.value)
            print(M)
            L = L+M
        print("Input for reduce function:\n", L)
        print("Output for reduce function:")
        Reducer_aw(L,aw_num.value)
output10 = widgets.Output()

aw_items = [aw_lower,aw_upper]
box = Box(children = aw_items, layout = layout)
display(aw_instruction,aw_sql, box, aw_num, submit_button, output10)
submit_button.on_click(clicked)

HTML(value='Show a list of counties in California that had more than the given number of wildfires during the …

HTML(value='select Counties, count(Name) from wildfire where month >= (lower) and month <= (upper) group by Co…

Box(children=(Dropdown(description='Month range:', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12), style=Desc…

IntText(value=0, description='number:', style=DescriptionStyle(description_width='initial'))

Button(description='Submit', style=ButtonStyle())

Output()